
## Importing data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('/content/filtered_data.csv')

In [3]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,2010,127 Hours,American,Danny Boyle,"James Franco, Amber Tamblyn, Kate Mara, Clémen...","biography, drama",https://en.wikipedia.org/wiki/127_Hours,Mountaineer Aron Ralston goes hiking at Utah's...
1,2010,The A-Team,American,Joe Carnahan,"Liam Neeson, Bradley Cooper, Quinton ""Rampage""...","action, adventure",https://en.wikipedia.org/wiki/The_A-Team_(film),"John ""Hannibal"" Smith is held captive in Mexic..."
2,2010,A Little Help,American,Michael J. Weithorn,"Jenna Fischer, Chris O'Donnell, Rob Benedict, ...",comedy,https://en.wikipedia.org/wiki/A_Little_Help,Laura Pehlke (Jenna Fischer) is married to Bob...
3,2010,Adventures of Power,American,Ari Gold,"Ari Gold, Michael McKean, Jane Lynch, Shoshann...",comedy,https://en.wikipedia.org/wiki/Adventures_of_Power,The story takes place in the fictional small t...
4,2010,Alice in Wonderland,American,Tim Burton,"Johnny Depp, Anne Hathaway, Helena Bonham Cart...","family, fantasy",https://en.wikipedia.org/wiki/Alice_in_Wonderl...,Troubled by a strange recurring dream and mour...


In [4]:
df = df[df["Release Year"] == 2017]

In [5]:
df.to_csv('test_data.csv', index=False)

In [6]:
len(df)

213

# LLM Experiment-1

In [1]:
%%capture
! pip install pyarrow==15.0 pylance lancedb bitsandbytes peft trl accelerate transformers langchain sentence-transformers langchainhub langchain_community chromadb langchain-huggingface

In [7]:
import os
import pandas as pd
from typing import List, Dict
from warnings import simplefilter
from langchain import hub
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, LanceDB
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from dotenv import load_dotenv

In [8]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [9]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
#repo_id = "tiiuae/falcon-7b-instruct"
# Set up a Hugging Face Endpoint to talk to the model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=2048, temperature=0.3, huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# Load the CSV data
loader = CSVLoader(file_path='/content/test_data.csv')
data = loader.load()

# Split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
            chunk_overlap=100,
            length_function=len,
            add_start_index=True,)
docs = text_splitter.split_documents(data)

In [11]:
docs[0]

Document(metadata={'source': '/content/test_data.csv', 'row': 0, 'start_index': 0}, page_content='Release Year: 2017\nTitle: Underworld: Blood Wars\nOrigin/Ethnicity: American\nDirector: Anna Foerster\nCast: Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Tobias Menzies, Lara Pulver, James Faulkner, Charles Dance\nGenre: action, horror\nWiki Page: https://en.wikipedia.org/wiki/Underworld:_Blood_Wars\nPlot: The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her to locate Eve, whose blood holds the key to building an army of vampire-werewolf hybrids.')

In [12]:
embed_fn = SentenceTransformerEmbeddings(model_name="all-minilm-l6-v2")
# Embed our data using the Embedding function and store it in a LanceDB database
db = Chroma.from_documents(docs, embed_fn)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
base_retriever = db.as_retriever(search_kwargs={"k" : 3})

In [14]:
relevant_docs = base_retriever.get_relevant_documents("Which movies involves vampires and lycans?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [15]:
relevant_docs

[Document(metadata={'row': 0, 'source': '/content/test_data.csv', 'start_index': 0}, page_content='Release Year: 2017\nTitle: Underworld: Blood Wars\nOrigin/Ethnicity: American\nDirector: Anna Foerster\nCast: Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Tobias Menzies, Lara Pulver, James Faulkner, Charles Dance\nGenre: action, horror\nWiki Page: https://en.wikipedia.org/wiki/Underworld:_Blood_Wars\nPlot: The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her to locate Eve, whose blood holds the key to building an army of vampire-werewolf hybrids.'),
 Document(metadata={'row': 169, 'source': '/content/test_data.csv', 'start_index': 0}, page_content='Release Year: 2017\nTitle: The Killing of a Sacred Deer\nOrigin/Ethnicity: American\nDirector: Yorgos Lanthimos\nCast: Yorgo

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the provided context, list the movies that match the query. If no relevant movies are found, respond with 'No Movies Found'. Provide the titles in the format: Movie_Title (Release Year). Do not provide any extra details regarding context or answer. Do not add any explanation for your answer.
### CONTEXT
{context}
### QUESTION
Question: {question}
### ANSWER
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [17]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough


retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [18]:
question = "Which movies involves vampires and lycans?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': '1. Underworld: Blood Wars (2017)\n2. It Comes at Night (2017)\n\nThese movies were found in the context provided and match the query.', 'context': [Document(metadata={'row': 0, 'source': '/content/test_data.csv', 'start_index': 0}, page_content='Release Year: 2017\nTitle: Underworld: Blood Wars\nOrigin/Ethnicity: American\nDirector: Anna Foerster\nCast: Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Tobias Menzies, Lara Pulver, James Faulkner, Charles Dance\nGenre: action, horror\nWiki Page: https://en.wikipedia.org/wiki/Underworld:_Blood_Wars\nPlot: The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her to locate Eve, whose blood holds the key to building an army of vampire-werewolf hybrids.'), Document(metadata={'row': 169, 'source': '/content/test_data.csv

### Groundtruth Dataset Creation

In [19]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [20]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [21]:
question_generation_llm = llm

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [22]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a film expert creating a quiz for advanced movie enthusiasts. For each context, create a question specific to the plot mentioned. Avoid generic questions. The questions should be such that the answer will always be the movie name.

Example questions:
- What are the best movies that explore themes of survival in the wilderness? OR
- In which movies does Main Actor belongs from Rich family and falls in love with a girl from poor background?

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
print(response)
output_dict = question_output_parser.parse(response)

  {
  "question": "Which movie's plot revolves around the remaining vampire covens trying to locate a woman whose blood holds the key to building an army of hybrid creatures, while the Lycans also search for her for different reasons?",
  "context": {
    "release_year": "2017",
    "title": "Underworld: Blood Wars",
    "origin": "American",
    "director": "Anna Foerster",
    "screenplay_by": "Cory Goodman",
    "cast": [
      "Anna Foerster (director)",
      "Cory Goodman (screenplay)",
      "Kate Beckinsale",
      "Theo James",
      "Tobias Menzies",
      "Lara Pulver",
      "James Faulkner",
      "Charles Dance"
    ],
    "genre": ["action", "horror"],
    "wiki_page": "https://en.wikipedia.org/wiki/Underworld:_Blood_Wars",
    "plot": "The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her t

In [23]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
Which movie's plot revolves around the remaining vampire covens trying to locate a woman whose blood holds the key to building an army of hybrid creatures, while the Lycans also search for her for different reasons?
context
{'release_year': '2017', 'title': 'Underworld: Blood Wars', 'origin': 'American', 'director': 'Anna Foerster', 'screenplay_by': 'Cory Goodman', 'cast': ['Anna Foerster (director)', 'Cory Goodman (screenplay)', 'Kate Beckinsale', 'Theo James', 'Tobias Menzies', 'Lara Pulver', 'James Faulkner', 'Charles Dance'], 'genre': ['action', 'horror'], 'wiki_page': 'https://en.wikipedia.org/wiki/Underworld:_Blood_Wars', 'plot': 'The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her to locate Eve, whose blood holds the key to building an army of vampire-werewolf hybrids.', 'metadata': {'sou

In [24]:
!pip install -q -U tqdm

In [25]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:15]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 15/15 [01:22<00:00,  5.49s/it]


In [26]:
qac_triples[5]

{'question': "In which movie does the protagonist, Selene, witness a series of blood memories after coming into contact with another character's blood?",
 'context': Document(metadata={'source': '/content/test_data.csv', 'row': 0, 'start_index': 3326}, page_content="While Selene is making her way through the castle, the guards in the dungeons are killed by Semira, who then escapes from her cell. Selene and David find Marius, but David is waylaid by Semira. As Marius and Selene's fight continues, a drop of Marius's blood lands on Selene's lips. She is suddenly flooded by a series of blood memories in which Marius finds Michael. She sees Marius capturing Michael and slitting his throat in order to collect his blood and consume it. Believing that Michael is dead, she bites her own wrist, using her own blood memories of the time she has spent with Michael, Eve, and David. Although Marius has transformed, Selene rips out his spine, killing him instantly. Semira drank Selene's blood that she

In [29]:
answer_generation_llm = llm

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a film expert creating a quiz for advanced movie enthusiasts. For each question and context, create an answer. Answer must be the movie name. If you don't know the answer, just say that you don't know.

answer: an answer (Movie Title) about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response)

In [30]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
Which movie's plot revolves around the remaining vampire covens trying to locate a woman whose blood holds the key to building an army of hybrid creatures, while the Lycans also search for her for different reasons?
context
{'Release Year': '2017', 'Title': 'Underworld: Blood Wars', 'Origin/Ethnicity': 'American', 'Director': 'Anna Foerster', 'Screenplay': 'Cory Goodman', 'Cast': ['Anna Foerster (director)', 'Cory Goodman (screenplay)', 'Kate Beckinsale', 'Theo James', 'Tobias Menzies', 'Lara Pulver', 'James Faulkner', 'Charles Dance'], 'Genre': ['action', 'horror'], 'Wiki Page': 'https://en.wikipedia.org/wiki/Underworld:_Blood_Wars', 'Plot': 'The remaining vampire covens are on the verge of being wiped out by the Lycans. Both species are searching for Selene: the vampires seeking justice for the deaths of Viktor and Marcus, and the Lycans, led by Marius, intending to use her to locate Eve, whose blood holds the key to building an army of vampire-werewolf hybrids.'}
answer
Und

In [31]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 13/13 [00:55<00:00,  4.28s/it]


In [32]:
output_dict

{'question': "In which movie does the protagonist have a deadly battle with the antagonist over a poison machine, resulting in the antagonist's death and the protagonist's fall into a hole?",
 'context': "Terravex gives chase up the mountain and the group escape by using the trucks to jump over the edge of the mountain. On the way, Rick helps Tripp and the group escape from Burke. After realizing the poison has been inserted, Tripp gets into a head-on battle with Burke, who attempts to push him into the drilling hole, but ends when Tripp and Creech overturn Burke's truck, destroying the poison machine and killing Burke when his truck is thrown into the equipment, only for Tripp to fall into the hole. Creech saves him from drowning before he and his parents depart back home and Terravex is exposed by the group for the experimentation that was harming the creatures' habitat. Tenneson is arrested, Tripp and Rick develop on good terms and together build a new engine for the truck, and Trip

### Creating Eval Dataset

In [33]:
!pip install -q -U datasets

In [34]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)

In [35]:
ground_truth_qac_set

,question,context,answer
0,Which movie's plot revolves around the remaini...,page_content='Release Year: 2017\nTitle: Under...,Underworld: Blood Wars
1,Which movie features a council member's betray...,"page_content='Semira, a council member of the ...",Underworld: Evolution
2,"In which movie does Selene, a vampire, reveal ...",page_content='The pair seeks refuge with the N...,Underworld: Evolution
3,"In the context of the Eastern Coven, which cha...",page_content='Upon Alexia's return to the East...,None
4,"In which vampire movie does the protagonist, S...",page_content='The coven comes under attack by ...,Underworld: Evolution
5,"In which movie does the protagonist, Selene, w...",page_content='While Selene is making her way t...,Underworld
6,Which Underworld film sees Selene reunited wit...,"page_content='In the aftermath, Selene, David,...",Underworld: Awakening
7,"In the 2017 American animated, adventure, and ...",page_content='Release Year: 2017\nTitle: Monst...,Amy Ryan: Voice of Meredith\nHolt McCallany: V...
8,"Which animation, adventure, sci-fi movie is ab...","page_content='Genre: animation, adventure, sci...",Monster Trucks
9,Which movie features a human befriending an oi...,"page_content='The next day Tripp, along with c...",NaN


In [36]:
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set

,question,context,answer
0,Which movie's plot revolves around the remaini...,Release Year: 2017\nTitle: Underworld: Blood W...,Underworld: Blood Wars
1,Which movie features a council member's betray...,"Semira, a council member of the Eastern Coven,...",Underworld: Evolution
2,"In which movie does Selene, a vampire, reveal ...",The pair seeks refuge with the Nordic Coven. T...,Underworld: Evolution
3,"In the context of the Eastern Coven, which cha...",Upon Alexia's return to the Eastern Coven's ca...,None
4,"In which vampire movie does the protagonist, S...",The coven comes under attack by Marius and his...,Underworld: Evolution
5,"In which movie does the protagonist, Selene, w...",While Selene is making her way through the cas...,Underworld
6,Which Underworld film sees Selene reunited wit...,"In the aftermath, Selene, David, and Lena are ...",Underworld: Awakening
7,"In the 2017 American animated, adventure, and ...",Release Year: 2017\nTitle: Monster Trucks\nOri...,Amy Ryan: Voice of Meredith\nHolt McCallany: V...
8,"Which animation, adventure, sci-fi movie is ab...","Genre: animation, adventure, sci-fi\nWiki Page...",Monster Trucks
9,Which movie features a human befriending an oi...,"The next day Tripp, along with classmate Mered...",NaN


In [37]:
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})
ground_truth_qac_set

,question,context,ground_truth
0,Which movie's plot revolves around the remaini...,Release Year: 2017\nTitle: Underworld: Blood W...,Underworld: Blood Wars
1,Which movie features a council member's betray...,"Semira, a council member of the Eastern Coven,...",Underworld: Evolution
2,"In which movie does Selene, a vampire, reveal ...",The pair seeks refuge with the Nordic Coven. T...,Underworld: Evolution
3,"In the context of the Eastern Coven, which cha...",Upon Alexia's return to the Eastern Coven's ca...,None
4,"In which vampire movie does the protagonist, S...",The coven comes under attack by Marius and his...,Underworld: Evolution
5,"In which movie does the protagonist, Selene, w...",While Selene is making her way through the cas...,Underworld
6,Which Underworld film sees Selene reunited wit...,"In the aftermath, Selene, David, and Lena are ...",Underworld: Awakening
7,"In the 2017 American animated, adventure, and ...",Release Year: 2017\nTitle: Monster Trucks\nOri...,Amy Ryan: Voice of Meredith\nHolt McCallany: V...
8,"Which animation, adventure, sci-fi movie is ab...","Genre: animation, adventure, sci-fi\nWiki Page...",Monster Trucks
9,Which movie features a human befriending an oi...,"The next day Tripp, along with classmate Mered...",NaN


In [37]:
temp_dict = ground_truth_qac_set.iloc[7]['ground_truth']
ground_truth_qac_set.iloc[7]['ground_truth'] = ''.join(temp_dict['canonical_answer'])

In [ ]:
ground_truth_qac_set.drop(columns=['metadata'], inplace=True)
ground_truth_qac_set

,question,context,ground_truth
0,Which movie's plot revolves around two warring...,Release Year: 2017\nTitle: Underworld: Blood W...,Underworld: Blood Wars
1,In which movie does the council member Semira ...,"Semira, a council member of the Eastern Coven,...",The movie 'Underworld: Evolution' is where the...
2,"In which movie does Selene, a vampire, reveal ...",The pair seeks refuge with the Nordic Coven. T...,Underworld: Evolution
3,In which movie does the betrayal of Alexia lea...,Upon Alexia's return to the Eastern Coven's ca...,The movie 'Blade: Trinity' provides the contex...
4,Which movie features a Nordic Coven that reviv...,The coven comes under attack by Marius and his...,"The Matrix: Reloaded. In this movie, Selene, t..."
5,"In which movie does the protagonist, after con...",While Selene is making her way through the cas...,The movie 'Underworld: Evolution' is the answe...
6,Which movie series introduces Selene as a new ...,"In the aftermath, Selene, David, and Lena are ...",Underworld
7,In the 2017 American animated adventure-sci-fi...,Release Year: 2017\nTitle: Monster Trucks\nOri...,"Holt McCallany is voiced by Holt McCallany, an..."
8,Which movie's plot revolves around a high scho...,"Genre: animation, adventure, sci-fi\nWiki Page...",Monster Trucks
9,In which movie does the protagonist befriend a...,"The next day Tripp, along with classmate Mered...",The movie 'Paul' (2011) is the answer to this ...


In [38]:
eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [39]:
# Convert the Dataset back to a Pandas DataFrame
eval_df = eval_dataset.to_pandas()

# Save the DataFrame to a CSV file
eval_df.to_csv('eval_dataset.csv', index=False)

## Evaluating RAG Pipeline

In [ ]:
from datasets import Dataset
eval_dataset = Dataset.from_csv("eval_dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [40]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 13
})

#### evaluating using RAGAS

In [41]:
!pip install -q -U ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.4 MB/s eta 0:00:00


In [42]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

In [43]:
def create_ragas_dataset(rag_pipeline, eval_dataset):
    rag_dataset = []
    for row in tqdm(eval_dataset):
        try:
            answer = rag_pipeline.invoke({"question": row["question"]})
            rag_dataset.append({
                "question": row["question"],
                "answer": answer["response"],
                "contexts": [context.page_content for context in answer["context"]],
                "ground_truth": row["ground_truth"]
            })
        except Exception as e:
            print(f"Error processing question: {row['question']}")
            print(f"Error: {str(e)}")
            # Optionally, you can add a placeholder or skip this entry

    rag_df = pd.DataFrame(rag_dataset)
    rag_eval_dataset = Dataset.from_pandas(rag_df)
    return rag_eval_dataset


"""def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"],
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset"""

'def create_ragas_dataset(rag_pipeline, eval_dataset):\n  rag_dataset = []\n  for row in tqdm(eval_dataset):\n    answer = rag_pipeline.invoke({"question" : row["question"]})\n    rag_dataset.append(\n        {"question" : row["question"],\n         "answer" : answer["response"],\n         "contexts" : [context.page_content for context in answer["context"]],\n         "ground_truths" : [row["ground_truth"]]\n         }\n    )\n  rag_df = pd.DataFrame(rag_dataset)\n  rag_eval_dataset = Dataset.from_pandas(rag_df)\n  return rag_eval_dataset'

In [44]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)

100%|██████████| 13/13 [00:34<00:00,  2.65s/it]


In [45]:
basic_qa_ragas_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 13
})

In [62]:
basic_qa_ragas_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 13
})

In [46]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

33056

In [47]:
def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

In [48]:
import nest_asyncio
nest_asyncio.apply()

In [49]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [50]:
basic_qa_result

{'context_precision': 0.5000, 'faithfulness': 0.5889, 'answer_relevancy': 0.7007, 'context_recall': 0.3846, 'answer_correctness': 0.5532, 'answer_similarity': 0.8615}

In [51]:
evaluation_df = basic_qa_result.to_pandas()
evaluation_df.head()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity
0,Which movie's plot revolves around the remaini...,1. Underworld: Blood Wars (2017),[Release Year: 2017\nTitle: Underworld: Blood ...,Underworld: Blood Wars,0.833333,1.000000,0.827069,1.0,0.879489,0.946496
1,Which movie features a council member's betray...,1. Underworld: Blood Wars (2016)\n2. Woodshock...,"[Ten years later, recently-hired nurse Elizabe...",Underworld: Evolution,0.500000,0.272727,0.863821,0.0,0.506799,0.827195
2,"In which movie does Selene, a vampire, reveal ...",Underworld: Blood Wars (2017),[The coven comes under attack by Marius and hi...,Underworld: Evolution,0.333333,1.000000,0.789070,0.0,0.225930,0.903835
3,"In the context of the Eastern Coven, which cha...",1. Semira (2000)\n2. Alexia (2001)\n\nNote: Th...,[Upon Alexia's return to the Eastern Coven's c...,None,0.000000,0.000000,0.787080,0.0,NaN,NaN
4,"In which vampire movie does the protagonist, S...",1. Underworld: Evolution (2006)\n2. Underworld...,[The coven comes under attack by Marius and hi...,Underworld: Evolution,0.833333,0.800000,0.916285,0.5,0.723685,0.894819


#### Creating standard pipeline to test other retreivers

In [54]:
def create_qa_chain(retriever):
  primary_qa_llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=2048, temperature=0.3, huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

  return created_qa_chain

#### Ensemble Retrieval

The basic idea is as follows:

1. Obtain User Question
2. Hit the Retriever Pair
    - Retrieve Documents with BM25 Sparse Vector Retrieval
    - Retrieve Documents with Dense Vector Retrieval Method
3. Collect and "fuse" the retrieved docs based on their weighting using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm into a single ranked list.
4. Use those documents to augment our generation.

Ensure your `weights` list - the relative weighting of each retriever - sums to 1!

In [52]:
!pip install -q -U rank_bm25

In [53]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(data)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2

vectorstore = Chroma.from_documents(docs, embed_fn)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25])

In [55]:
ensemble_retriever_qa_chain = create_qa_chain(ensemble_retriever)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [57]:
ensemble_retriever_qa_chain.invoke({"question" : "Which movies involves vampires and lycans?"})["response"]

"1. Underworld: Blood Wars (2017)\n2. The Killing of a Sacred Deer (2017)\n\nPlease note that while 'Underworld: Blood Wars' does involve vampires and lycans, 'The Killing of a Sacred Deer' does not have these creatures in its plot. However, the question does not ask for a detailed analysis of the movies, only their titles. Therefore, both movies are included in the answer."

In [58]:
ensemble_qa_ragas_dataset = create_ragas_dataset(ensemble_retriever_qa_chain, eval_dataset)

  8%|▊         | 1/13 [00:00<00:01,  6.45it/s]

Error processing question: Which movie's plot revolves around the remaining vampire covens trying to locate a woman whose blood holds the key to building an army of hybrid creatures, while the Lycans also search for her for different reasons?
Error: 502 Server Error: Bad Gateway for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2


100%|██████████| 13/13 [00:20<00:00,  1.56s/it]


In [59]:
ensemble_qa_result = evaluate_ragas_dataset(ensemble_qa_ragas_dataset)

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[17]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[53]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[52]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[16]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [60]:
ensemble_qa_result

{'context_precision': 0.6806, 'faithfulness': 0.5507, 'answer_relevancy': 0.7531, 'context_recall': 0.6389, 'answer_correctness': 0.5630, 'answer_similarity': 0.8620}

#### Parent Document Retriever

One of the easier ways we can imagine improving a retriever is to embed our documents into small chunks, and then retrieve a significant amount of additional context that "surrounds" the found context.

The basic outline of this retrieval method is as follows:

Obtain User Question
Retrieve child documents using Dense Vector Retrieval
Merge the child documents based on their parents. If they have the same parents - they become merged.
Replace the child documents with their respective parent documents from an in-memory-store.
Use the parent documents to augment generation.

In [98]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100, length_function=len)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100, length_function=len)

vectorstore = Chroma(collection_name="split_parents", embedding_function=embed_fn)

store = InMemoryStore()

In [99]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [100]:
parent_document_retriever.add_documents(data)

In [101]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [102]:
parent_document_retriever_qa_chain.invoke({"question" : "Which movies involves vampires and lycans?"})["response"]

'- Underworld: Blood Wars (2017)'

In [103]:
pdr_qa_ragas_dataset = create_ragas_dataset(parent_document_retriever_qa_chain, eval_dataset)

100%|██████████| 13/13 [00:14<00:00,  1.08s/it]


In [104]:
pdr_qa_result = evaluate_ragas_dataset(pdr_qa_ragas_dataset)

Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [105]:
pdr_qa_result

{'context_precision': 0.7115, 'faithfulness': 0.7269, 'answer_relevancy': 0.6310, 'context_recall': 0.6282, 'answer_correctness': 0.5041, 'answer_similarity': 0.8578}

#### Comparing all the results

In [106]:
def create_df_dict(pipeline_name, pipeline_items):
  df_dict = {"name" : pipeline_name}
  for name, score in pipeline_items:
    df_dict[name] = score
  return df_dict

In [107]:
basic_rag_df_dict = create_df_dict("basic_rag", basic_qa_result.items())

In [108]:
ensemble_rag_df_dict = create_df_dict("ensemble_rag", ensemble_qa_result.items())

In [109]:
pdr_rag_df_dict = create_df_dict("pdr_rag", pdr_qa_result.items())

In [110]:
results_df = pd.DataFrame([basic_rag_df_dict, pdr_rag_df_dict, ensemble_rag_df_dict])

In [111]:
results_df.sort_values("answer_correctness", ascending=False)

,name,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity
2,ensemble_rag,0.680556,0.550694,0.753114,0.638889,0.563016,0.862046
0,basic_rag,0.500000,0.588928,0.700673,0.384615,0.553154,0.861519
1,pdr_rag,0.711538,0.726923,0.630971,0.628205,0.504062,0.857823


## Experimenting with other LLMS and Parameters

In [1]:
%%capture
! pip install pyarrow==15.0 pylance lancedb bitsandbytes peft trl accelerate transformers langchain sentence-transformers langchainhub langchain_community chromadb langchain-huggingface ragas tqdm rank_bm25 datasets

In [1]:
import os
import pandas as pd
from typing import List, Dict
from warnings import simplefilter
from langchain import hub
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, LanceDB
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from dotenv import load_dotenv

In [9]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate
from datasets import Dataset

In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from tqdm import tqdm

In [2]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
df = pd.read_csv("/content/filtered_data.csv")
df = df[df["Release Year"] == 2017]
len(df)

213

In [5]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
1388,2017,Underworld: Blood Wars,American,Anna Foerster,Anna Foerster (director); Cory Goodman (screen...,"action, horror",https://en.wikipedia.org/wiki/Underworld:_Bloo...,The remaining vampire covens are on the verge ...
1389,2017,Monster Trucks,American,Chris Wedge,"Chris Wedge (director); Jonathan Aibel, Glenn ...","animation, adventure, sci-fi",https://en.wikipedia.org/wiki/Monster_Trucks_(...,Terravex Oil is in the midst of a fracking ope...
1390,2017,The Bye Bye Man,American,Stacy Title,Stacy Title (director); Jonathan Penner (scree...,horror,https://en.wikipedia.org/wiki/The_Bye_Bye_Man,"In 1969, a mass murder occurs in which a man k..."
1391,2017,Sleepless,American,Baran bo Odar,Baran bo Odar (director); Andrea Berloff (scre...,"action, thriller",https://en.wikipedia.org/wiki/Sleepless_(2017_...,"In Las Vegas, vice LVMPD policemen Vincent Dow..."
1392,2017,100 Streets,American,Jim O'Hanlon,Jim O'Hanlon (director); Leon F. Butler (scree...,drama,https://en.wikipedia.org/wiki/100_Streets,The film centers on three characters who have ...


In [6]:
df.to_csv("test_data.csv")

Hyper-parameters to test
- LLM (repo_id)
- temperature
- Embedding_function (embed_fn)
- Retreiver (retreiver)
- chunk_overlap
- chunk_size
- Top K sampling (top_k)


In [57]:
def rag_experiment_retreiver(csv_file_path, embed_fn, chunk_overlap, chunk_size, top_k):
  loader = CSVLoader(file_path=csv_file_path)
  data = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(data)
  bm25_retriever = BM25Retriever.from_documents(docs)
  bm25_retriever.k = top_k

  vectorstore = Chroma.from_documents(docs, embed_fn)
  chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

  ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25])
  return vectorstore, ensemble_retriever

In [31]:
def create_qa_chain(llm, retriever):
  primary_qa_llm = llm
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

  return created_qa_chain

def create_ragas_dataset(rag_pipeline, eval_dataset):
    rag_dataset = []
    for row in tqdm(eval_dataset):
        try:
            answer = rag_pipeline.invoke({"question": row["question"]})
            rag_dataset.append({
                "question": row["question"],
                "answer": answer["response"],
                "contexts": [context.page_content for context in answer["context"]],
                "ground_truth": row["ground_truth"]
            })
        except Exception as e:
            print(f"Error processing question: {row['question']}")
            print(f"Error: {str(e)}")
            # Optionally, you can add a placeholder or skip this entry

    rag_df = pd.DataFrame(rag_dataset)
    rag_eval_dataset = Dataset.from_pandas(rag_df)
    return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

In [32]:
def rag_experiment_pipeline(repo_id, temperature, retriever, eval_dataset):
  llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=2048, temperature=temperature, huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)
  template = """Based on the provided context, list the movies that match the query. If no relevant movies are found, respond with 'No Movies Found'. Provide the titles in the format: Movie_Title (Release Year). Do not provide any extra details regarding context or answer. Do not add any explanation for your answer.
  ### CONTEXT
  {context}
  ### QUESTION
  Question: {question}
  ### ANSWER
  Answer:
  """

  prompt = ChatPromptTemplate.from_template(template)
  ensemble_retriever_qa_chain = create_qa_chain(llm, ensemble_retriever)
  ensemble_qa_ragas_dataset = create_ragas_dataset(ensemble_retriever_qa_chain, eval_dataset)
  ensemble_qa_result = evaluate_ragas_dataset(ensemble_qa_ragas_dataset)
  return ensemble_qa_result

In [33]:
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
#repo_id = "tiiuae/falcon-7b-instruct"
repo_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [11]:
embed_fn = SentenceTransformerEmbeddings(model_name="all-minilm-l6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
eval_dataset = Dataset.from_csv("eval_dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [34]:
csv_file_path = "/content/test_data.csv"
chunk_overlap = 100
chunk_size = 1000
top_k = 2

In [35]:
 test_vectorestore, test_retreiver = rag_experiment_retreiver(csv_file_path, embed_fn, chunk_overlap, chunk_size, top_k)

In [36]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.1"
temperature = 0.3

In [37]:
test_result = rag_experiment_pipeline(repo_id=repo_id, temperature=temperature, retriever=test_retreiver, eval_dataset=eval_dataset)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 13/13 [00:11<00:00,  1.09it/s]


Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [38]:
test_result

{'context_precision': 0.7564, 'faithfulness': 0.4872, 'answer_relevancy': 0.8142, 'context_recall': 0.5641, 'answer_correctness': 0.4437, 'answer_similarity': 0.8519}

In [39]:
def create_df_dict(model_name, pipeline_items):
  df_dict = {"name" : model_name}
  for name, score in pipeline_items:
    df_dict[name] = score
  return df_dict

In [44]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
mistral_02_result = rag_experiment_pipeline(repo_id=repo_id, temperature=temperature, retriever=test_retreiver, eval_dataset=eval_dataset)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 13/13 [00:15<00:00,  1.19s/it]


Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [51]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
mistral_7b_03_result = rag_experiment_pipeline(repo_id=repo_id, temperature=temperature, retriever=test_retreiver, eval_dataset=eval_dataset)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 13/13 [00:15<00:00,  1.18s/it]


Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


In [47]:
ensemble_qa_result

{'context_precision': 0.6756, 'faithfulness': 0.6538, 'answer_relevancy': 0.8221, 'context_recall': 0.5897, 'answer_correctness': 0.4108, 'answer_similarity': 0.8642}

In [50]:
mistral_02_result

{'context_precision': 0.6667, 'faithfulness': 0.5641, 'answer_relevancy': 0.6566, 'context_recall': 0.6026, 'answer_correctness': 0.6650, 'answer_similarity': 0.8722}

In [52]:
mistral_7b_03_result

{'context_precision': 0.7436, 'faithfulness': 0.4487, 'answer_relevancy': 0.6959, 'context_recall': 0.5641, 'answer_correctness': 0.5990, 'answer_similarity': 0.8715}

In [54]:
rag_df_dict_1 = create_df_dict("mistral_7b_0.1", test_result.items())
rag_df_dict_2 = create_df_dict("tiiuae/falcon-7b-instruct", ensemble_qa_result.items())
rag_df_dict_3 = create_df_dict("mistral_7b_0.2", mistral_02_result.items())
rag_df_dict_4 = create_df_dict("mistral_7b_0.3", mistral_7b_03_result.items())

results_df = pd.DataFrame([rag_df_dict_1, rag_df_dict_2, rag_df_dict_3, rag_df_dict_4])
results_df.sort_values("answer_correctness", ascending=False)

,name,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity
2,mistral_7b_0.2,0.666667,0.564103,0.656640,0.602564,0.665005,0.872168
3,mistral_7b_0.3,0.743590,0.448718,0.695918,0.564103,0.598965,0.871498
0,mistral_7b_0.1,0.756410,0.487179,0.814223,0.564103,0.443661,0.851915
1,tiiuae/falcon-7b-instruct,0.675641,0.653846,0.822063,0.589744,0.410751,0.864208


In [55]:
results_df.to_csv("results_df.csv")

In [59]:
# Experiment Parameters
experiment_params = {
    "repo_id": ["mistralai/Mistral-7B-Instruct-v0.2"],
    "temperature": [0.3, 0.5],
    "embed_fn": [SentenceTransformerEmbeddings(model_name="all-minilm-l6-v2")],
    "chunk_overlap": [100, 200],
    "chunk_size": [1000, 2000],
    "top_k": [2, 5]
}

# Create Experimental Pipeline
results = []
for repo_id in experiment_params["repo_id"]:
    for temperature in experiment_params["temperature"]:
        for embed_fn in experiment_params["embed_fn"]:
            for chunk_overlap in experiment_params["chunk_overlap"]:
                for chunk_size in experiment_params["chunk_size"]:
                    for top_k in experiment_params["top_k"]:
                        vectorstore, retriever = rag_experiment_retreiver(csv_file_path, embed_fn, chunk_overlap, chunk_size, top_k)
                        result = rag_experiment_pipeline(repo_id, temperature, retriever, eval_dataset)
                        df_dict = create_df_dict(repo_id, result.items())
                        df_dict["temperature"] = temperature
                        df_dict["chunk_overlap"] = chunk_overlap
                        df_dict["chunk_size"] = chunk_size
                        df_dict["top_k"] = top_k
                        results.append(df_dict)

experiment_df = pd.DataFrame(results)
print(experiment_df)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 13/13 [00:15<00:00,  1.21s/it]


Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


...

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


100%|██████████| 13/13 [00:01<00:00,  8.88it/s]


Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[23]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[59]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[22]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')
ERROR:ragas.executor:Exception raised in Job[58]: TypeError(argument 'text': 'NoneType' object cannot be converted to 'PyString')


                                  name  context_precision  faithfulness  \
0   mistralai/Mistral-7B-Instruct-v0.2           0.717949      0.500500   
1   mistralai/Mistral-7B-Instruct-v0.2           0.692308      0.449217   
2   mistralai/Mistral-7B-Instruct-v0.2           0.743590      0.535664   
3   mistralai/Mistral-7B-Instruct-v0.2           0.743590      0.505750   
4   mistralai/Mistral-7B-Instruct-v0.2           0.743590      0.505750   
5   mistralai/Mistral-7B-Instruct-v0.2           0.666667      0.492929   
6   mistralai/Mistral-7B-Instruct-v0.2           0.730769      0.600622   
7   mistralai/Mistral-7B-Instruct-v0.2           0.653846      0.653846   
8   mistralai/Mistral-7B-Instruct-v0.2           0.730769      0.562454   
9   mistralai/Mistral-7B-Instruct-v0.2           0.653846      0.543223   
10  mistralai/Mistral-7B-Instruct-v0.2           0.730769      0.549634   
11  mistralai/Mistral-7B-Instruct-v0.2           0.653846      0.572711   
12  mistralai/Mistral-7B-

In [60]:
experiment_df.sort_values("answer_correctness", ascending=False)

,name,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity,temperature,chunk_overlap,chunk_size,top_k
13,mistralai/Mistral-7B-Instruct-v0.2,0.653846,0.536081,0.721297,0.592308,0.780237,0.881119,0.5,200,1000,5
8,mistralai/Mistral-7B-Instruct-v0.2,0.730769,0.562454,0.785155,0.613462,0.750562,0.877488,0.5,100,1000,2
11,mistralai/Mistral-7B-Instruct-v0.2,0.653846,0.572711,0.720085,0.569231,0.749546,0.880612,0.5,100,2000,5
10,mistralai/Mistral-7B-Instruct-v0.2,0.730769,0.549634,0.783472,0.528846,0.746843,0.877513,0.5,100,2000,2
14,mistralai/Mistral-7B-Instruct-v0.2,0.730769,0.587363,0.722215,0.613462,0.741374,0.882579,0.5,200,2000,2
12,mistralai/Mistral-7B-Instruct-v0.2,0.730769,0.577106,0.722247,0.619231,0.738768,0.880613,0.5,200,1000,2
15,mistralai/Mistral-7B-Instruct-v0.2,0.743590,0.548901,0.721918,0.575000,0.709386,0.882564,0.5,200,2000,5
9,mistralai/Mistral-7B-Instruct-v0.2,0.653846,0.543223,0.785629,0.519231,0.699101,0.877504,0.5,100,1000,5
1,mistralai/Mistral-7B-Instruct-v0.2,0.692308,0.449217,0.589856,0.576923,0.696879,0.876392,0.3,100,1000,5
6,mistralai/Mistral-7B-Instruct-v0.2,0.730769,0.600622,0.603612,0.592308,0.657765,0.874545,0.3,200,2000,2


In [61]:
experiment_df.to_csv("rag_evaluation_experiment_df.csv")